# Better performance with tf.function


In TensorFlow 2.0, eager execution is turned on by default. The user interface is intuitive and flexible (running one-off operations is much easier
and faster), but this can come at the expense of performance and deployability.

To get peak performance and to make your model deployable anywhere, use
`tf.function` to make graphs out of your programs.
Thanks to AutoGraph, a surprising amount of Python code just works with
tf.function, but there are still pitfalls to be wary of.

The main takeaways and recommendations are:

- Don't rely on Python side effects like object mutation or list appends.
- tf.function works best with TensorFlow ops, rather than NumPy ops or Python primitives.
- When in doubt, use the `for x in y` idiom.

In [1]:
import tensorflow as tf

Define a helper function to demonstrate the kinds of errors you might encounter:

In [2]:
import traceback
import contextlib

# Some helper code to demonstrate the kinds of errors you might encounter.
@contextlib.contextmanager
def assert_raises(error_class):
    try:
        yield
    except error_class as e:
        print('Caught expected exception \n  {}:'.format(error_class))
        traceback.print_exc(limit=2)
    except Exception as e:
        raise e
    else:
        raise Exception('Expected {} to be raised but no error was raised!'.format(error_class))


## Basics

A `tf.function` you define is just like a core TensorFlow operation: You can execute it eagerly; you can use it in a graph; it has gradients; and so on.

In [7]:
@tf.function 
def add(a,b):
    return a+b
add(tf.zeros([2,2]),tf.ones([2,2]))

<tf.Tensor: id=28, shape=(2, 2), dtype=float32, numpy=
array([[1., 1.],
       [1., 1.]], dtype=float32)>

In [10]:
v = tf.Variable(1.0)
with tf.GradientTape() as t:
    result = add(v,1.0)
t.gradient(result,v)

<tf.Tensor: id=59, shape=(), dtype=float32, numpy=1.0>

In [11]:
result

<tf.Tensor: id=57, shape=(), dtype=float32, numpy=2.0>

You can use functions inside functions

In [12]:
@tf.function
def dense_layer(x, w, b):
    return add(tf.matmul(x, w), b)

dense_layer(tf.ones([3, 2]), tf.ones([2, 2]), tf.ones([2]))

<tf.Tensor: id=85, shape=(3, 2), dtype=float32, numpy=
array([[3., 3.],
       [3., 3.],
       [3., 3.]], dtype=float32)>

<a id="tracing"></a>

## Tracing and polymorphism

Python's dynamic typing means that you can call functions with a variety of argument types, and Python will do something different in each scenario.

On the other hand, TensorFlow graphs require static dtypes and shape dimensions. `tf.function` bridges this gap by retracing the function when necessary to generate the correct graphs. Most of the subtlety of `tf.function` usage stems from this retracing behavior.

You can call a function with arguments of different types to see what is happening.

In [13]:
# Functions are polymorphic

@tf.function
def double(a):
    print("Tracing with", a)
    return a + a

print(double(tf.constant(1)))
print()
print(double(tf.constant(1.1)))
print()
print(double(tf.constant("a")))
print()


Tracing with Tensor("a:0", shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)

Tracing with Tensor("a:0", shape=(), dtype=float32)
tf.Tensor(2.2, shape=(), dtype=float32)

Tracing with Tensor("a:0", shape=(), dtype=string)
tf.Tensor(b'aa', shape=(), dtype=string)



To control the tracing behavior, use the following techniques:

- Create a new `tf.function`. Separate `tf.function` objects are guaranteed not to share traces. 
- Use the `get_concrete_function` method to get a specific trace
- Specify `input_signature` when calling `tf.function` to trace only once per calling graph.

In [14]:
print("Obtaining concrete trace")
double_strings = double.get_concrete_function(tf.TensorSpec(shape=None, dtype=tf.string))
print("Executing traced function")
print(double_strings(tf.constant("a")))
print(double_strings(a=tf.constant("b")))
print("Using a concrete trace with incompatible types will throw an error")
with assert_raises(tf.errors.InvalidArgumentError):
    double_strings(tf.constant(1))

Obtaining concrete trace
Tracing with Tensor("a:0", dtype=string)
Executing traced function
tf.Tensor(b'aa', shape=(), dtype=string)
tf.Tensor(b'bb', shape=(), dtype=string)
Using a concrete trace with incompatible types will throw an error
Caught expected exception 
  <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>:


Traceback (most recent call last):
  File "<ipython-input-2-af6dbe3f6b0e>", line 8, in assert_raises
    yield
  File "<ipython-input-14-0174c36f77e9>", line 8, in <module>
    double_strings(tf.constant(1))
tensorflow.python.framework.errors_impl.InvalidArgumentError: cannot compute __inference_double_112 as input #0(zero-based) was expected to be a string tensor but is a int32 tensor [Op:__inference_double_112]
